# Speaker Identification with Ey and Breath
> Summary
This project explores the possibilities and the methodologies to identify speakers' identities from short recordings of *`Ey` sounds* and *`breath`es*. We construct two neural networks to capture the features of sampled recordings and to predict the speaker identities. One is `conv_net + lstm`, and the other is `conv_net + tdnn`.
===

## Goals
Our goals are
1. extract features from recordings;
2. find ways to predict speaker's identity using these features, and prove their effectiveness.

## Backgrounds

## Methods
Looking at the `spectrogram` of the recordings, we observe that

[comment]: # ([pic_spectrogram]: ./drafts/images/spectrogram.jpg "Spectrogram")
1. different speakers have different harmonics and patterns;
2. a speaker having a higher pitch would have similar patterns with a different person in `log-spectrogram`;
3. the spectrograms of a same person with the same sounds have varied lengths (time durations).

In order to account for the scale invariance and the shift invariance, we propose
1. use `constant Q transform` to extract spectrograms from recordings;
2. use `convolution network` to extract features along the frequencies;
3. use `recurrent network` (Long Short-Term Memory in this task) / `Time Delay Neural Network` (TDNN) to capture the dynamics along the time steps.

### Constant Q transform
It is closely relate to the Fourier transfrom. It has series of logarithmically spaced filters, making it suitable for analysing music signals. The key components of constant Q transform are
1. The center frequencies are $f_k = 2^{\frac{k}{b}}f_0$, where $b$ is the number of filters per oactave, $k = 1, \ldots, K$, and the total number of frequency bins $K = b\log_2(\frac{f_{max}}{f_0})$.
2. The bandwidth of the $k$-th filter is $\delta f_k = f_{k+1} - f_{k} = f_k (2^{\frac{1}{b}} - 1)$. The ratio of frequency to bandwidth is constant $Q = \frac{f_k}{\delta f_k} = \frac{1}{(2^{\frac{1}{b}} - 1)}$, and hence the notation `constant Q`.
3. The window length for the $k$-th frequency bin is $N_k = Q\frac{f_s}{f_k}$, where $f_s$ is the sampling frequency.
4. Finally, the constant Q transform
$$ x[k] = \frac{1}{N_k}\sum_{n=0}^{N_k-1}{x[n]w_{N_k}[n]}e^{-j2\pi Qn/N_k}$$

### Long Short-Term Memory
The LSTM is a variant of Recurrent Neural Networks (RNN). It itself has many variants too. It was originally proposed by `Sepp Hochreiter and Jürgen Schmidhuber` in 1997, and then is used to effectively model contextual sequences. It is designed with internal memory cells and controlling gates to selectively memorize states and fight vanishing or exploding gradients. See the structure of a LSTM cell,![lstm_cell](./drafts/images/lstm_cell.png) and a sequence of cell units ![lstm_seq](./drafts/images/lstm_seq.png)

####  Forward propagation
The forward pass
at time $t$
1. Forget gate: $f_t = \sigma(W_f x_t + U_f h_{t-1} + b_f) = \sigma(\hat{f}^t)$
2. Input gate: $i_t = \sigma(W_i x_t + U_i h_{t-1} + b_i) = \sigma(\hat{i}^t)$
and candidate input: $a_t = tanh(W_c x_t + U_c h_{t-1} + b_c) = tanh(\hat{a}^t)$
3. Cell state update: $C_t = f_t C_{t-1} + i_t a_t$
4. Output gate: $o_t = \sigma(W_o x_t + U_o h_{t-1} + b_o) = \sigma(\hat{o}^t)$

Let $z^t = [\hat{a}^t\, \hat{i}^t\, \hat{f}^t\, \hat{o}^t]$
5. Hidden output: $h_t = o_t tanh(C_t)$

#### Backpropagation through time
The error is backpropagated and the gradients are calculated to update the weights. Different with multilayer perception networks, we need to backpropagate the gradients through time (BPTT). ![bptt](./drafts/images/bptt.png) Using the chain rule, we unroll the network at each time step and compute the partial derivatives of the error wrt each paramter participated in. 
1. $$\frac{\partial E}{\partial o_i^t} = \frac{\partial E}{\partial h_i^t}\frac{\partial h_i^t}{o_i^t} = \delta h_i^t tanh(c_i^t)$$
$$\delta o^t = \delta h^t tanh(c^t)$$
2. $$\frac{\partial E}{\partial c_i^t} = \frac{\partial E}{\partial h_i^t}\frac{\partial h_i^t}{c_i^t} = \delta h_i^t o_i^t (1 - tanh^2(c_i^t))$$
$$\delta c^t += \delta h^t o^t (1 - tanh^2(c^t))$$
3. $$\frac{\partial E}{\partial i_i^t} = \frac{\partial E}{\partial c_i^t}\frac{\partial c_i^t}{i_i^t} = \delta c_i^t a_i^t$$
$$\delta i^t = \delta c^t a^t$$
4. $$\frac{\partial E}{\partial f_i^t} = \frac{\partial E}{\partial c_i^t}\frac{\partial c_i^t}{f_i^t} = \delta c_i^t c_i^{t-1}$$
$$\delta f^t = \delta c^t c^{t-1}$$
5. $$\frac{\partial E}{\partial a_i^t} = \frac{\partial E}{\partial c_i^t}\frac{\partial c_i^t}{a_i^t} = \delta c_i^t i_i^t$$
$$\delta a^t = \delta c^t i^t$$
6. $$\frac{\partial E}{\partial c_i^{t-1}} = \frac{\partial E}{\partial c_i^t}\frac{\partial c_i^t}{c_i^{t-1}} = \delta c_i^t f_i^t$$
$$\delta c^{t-1} = \delta c^t f^t$$
7. $$\delta \hat{a}^t = \delta a^t (1 - tanh^2(\hat{a}^t))$$
$$\delta \hat{i}^t = \delta i^t i^t (1 - i^t)$$
$$\delta \hat{f}^t = \delta f^t f^t (1 - f^t)$$
$$\delta \hat{o}^t = \delta o^t o^t (1 - o^t)$$
$$\delta z^t = [\delta \hat{a}^t\, \delta \hat{i}^t\, \delta \hat{f}^t\, \delta \hat{o}^t]$$
8. $$\delta I^t = W \delta z^t,$$ where $I^t = [x^t\, h^{t-1}]$.
$$\delta W^t = \delta z^t I^t$$

Given $x = [x_1, \ldots, x_T]$, $\delta W = \sum_{t=1}^{T} \delta W^t$.

## Implementation
### Data Preparation
#### Feature extraction
* num_bin_per_octave = 48
* fs = 44100 Hz
* fmin = 27.5 Hz
* fmax = fs/2

#### Dataset
We selected from the dataset the portion in which each speaker has more than **100** samples. This contains **12942** instances out of a total **25368** instances, corresponding to **53** speakers out of a total **805** speakers. Then, for each speaker, we select 70% as training set, 20% as validation set, and 10% as test set. The datasets are the shuffled. The codes are shown below.

In [2]:
"""
Data preparation for speaker identification with Ey Breath.
"""
from __future__ import print_function
import os
import sys

import numpy as np
from theano import config

speaker_dict = dict() # store 'speaker_id (string) : id (integer)' pairs
instance_dict = dict() # store 'speaker_id (string) : instances (list of
    # instance objects)' pairs
count_dict = dict() # store 'speaker_id (string) : counter (integer)' pairs
instance_collection = [] # collection of instances

# TODO:
def normalization():
    """
    Normalize input features.
    """
    pass

class Instance:
    """
    One training instance.
    """
    def __init__(self, file_id, speaker_id, featvec):
        self.file_id = file_id
        self.speaker_id = speaker_id
        self.featvec = featvec

def read_instance(data_path, filename, spk_smpl_thrd = 100, wrt_dict = False):
    """
    Read data as instances.

    :data_path: (string) path containing data to be processed
    :filename: (string) file containing names of interested files
    :spk_smpl_thrd: (integer) only speaker that has samples exceeds
    this number is counted in
    :wrt_dict: (bool) write speaker_dict to file or not
    """

    filelist = [l for l in open(filename)] # all files in filename
    interest_filelist = [''.join([ l.split()[0].split('.')[0], '.txt' ])
                         for l in filelist] # filenames of interest
    num_files = len(filelist)
    for i in xrange(num_files):
        name = filelist[i]
        sspk_id = name.split()[1]
        if sspk_id in speaker_dict: # if speaker in dict
            count_dict[sspk_id] += 1 # add counter
            speaker_id = speaker_dict[sspk_id] # get mapped id

        else: # if speaker shows up first time
            count_dict[sspk_id] = 1 # set counter
            speaker_dict[sspk_id] = len(speaker_dict)
            speaker_id = speaker_dict[sspk_id]
            instance_dict[sspk_id] = [] # init speaker instance list

        file = interest_filelist[i]
        featvec = np.asarray([
            line.strip().split(',')
            for line in open(os.path.join(data_path, file))
        ], dtype = 'float')

        instance_dict[sspk_id].append( Instance(file, speaker_id, featvec) )

    enrolled_ins_count = 0 # counter for enrolled instances
    sid = 0
    for k in count_dict: # for each speaker
        if count_dict[k] >= spk_smpl_thrd: # if exceeds threshold
            ins_list = instance_dict[k]
            enrolled_ins_count += len(ins_list)
            for r in ins_list:
                ins_list[r].speaker_id = int(sid) # reprog speaker id
            instance_collection.append(ins_list) # add instances to collection
            sid += 1
    num_enrolled_spk = len(instance_collection) # number of enrolled speakers

    if wrt_dict:
        write_dict(speaker_dict, ''.join([filename, '_consq_speaker_dict.txt']))
        print('Saved speaker_dict to file: ', filename + '_consq_speaker_dict.txt')

    print('processed ', filename, ', total ', num_files, ' files')
    print('total ', len(speaker_dict), ' speakers')
    print('totally enrolled ', enrolled_ins_count, ' instances, ',
          num_enrolled_spk, ' speakers.')

    return num_enrolled_spk, enrolled_ins_count

def write_dict(speaker_dict, filename):
    out = ''
    for k in speaker_dict:
        out += str(k) + ',' + str(speaker_dict[k]) + '\n' # k, v
    with open(filename, 'w') as f:
        f.write(out)

def prepare_data(ins_list, idx_list):
    """
    Format data to network required form.
    :ins_list: list of instances
    :idx_list: list of indices
    <- [X (list of 4d tensors), y (list of integers)]
    """

    X = []
    y = []
    for idx in idx_list:
        ins = ins_list[idx]
        spk_id = ins.speaker_id
        y.append(spk_id)
        ins_vec = ins.featvec
        nrow, ncol = ins_vec.shape
        inp = np.asarray(ins_vec, dtype = config.floatX).reshape(
            (1, 1, nrow, ncol))
        X.append(inp)

    return zip(X, y)

def load_data(dpath, filename, shuffle = False, spk_smpl_thrd = 100):
    """
    Load data.
    <- train_set, val_set, test_set: [X (list of 4d tensors), y (list of integers)]
    """
    # Read instances from constant Q features
    num_enrolled_spk, enrolled_ins_count = read_instance(dpath, filename, spk_smpl_thrd)

    # Split dataset
    train_set = []
    val_set = []
    test_set = []
    split_ratio = (0.7, 0.2, 0.1)
    for i in xrange(num_enrolled_spk):
        ins_list = instance_collection[i]
        num_ins = len(ins_list)
        num_trns = int(np.floor(num_ins * split_ratio[0]))
        num_vals = int(np.floor(num_ins * split_ratio[1]))
        num_devs = int(num_ins - num_trns - num_vals)

        smpl_list = np.arange(num_ins)

        if shuffle:
            np.random.shuffle(smpl_list)

        trn_list = smpl_list[: num_trns]
        val_list = smpl_list[num_trns : num_trns + num_vals]
        dev_list = smpl_list[num_trns + num_vals : num_ins]

        xy_trn = prepare_data(ins_list, trn_list)
        xy_val = prepare_data(ins_list, val_list)
        xy_dev = prepare_data(ins_list, dev_list)

        for x, y in xy_trn:
            train_set.append((x, y))
        for x, y in xy_val:
            val_set.append((x, y))
        for x, y in xy_dev:
            test_set.append((x, y))

    if shuffle:
        tlf = np.random.permutation(len(train_set))
        vlf = np.random.permutation(len(val_set))
        dlf = np.random.permutation(len(test_set))
        train_set = [train_set[s] for s in tlf]
        val_set = [val_set[s] for s in vlf]
        test_set = [test_set[s] for s in dlf]

    return train_set, val_set, test_set

In [3]:
# Usage
dpath = './feat_constq'
train_set, val_set, test_set = load_data(os.path.join(dpath ,'ey'), './ey.interested', shuffle = True, spk_smpl_thrd = 100)

TypeError: object cannot be interpreted as an index

### Network Structure
The structures of `conv_net + lstm` and `conv_net + tdnn`
![lstm](./drafts/images/lstm.png)
![lstm](./drafts/images/tdnn.png)


In [ ]:
# Core layers
# LSTM
class LSTMBuilder:
    """
    Net builder for LSTM.
    """
    def __init__(self, inp, prefix, rand_scheme = 'standnormal'):
        """
        :inp: (num_time_steps, num_samples, num_embedding_size)

        TODO:
        1. Batch & mask
        2. Output dim
        """

        n_tstep, n_samp, embd_size = inp.shape[0], inp.shape[1], inp.shape[2]
        embd_size = 15
        self.n_tstep = n_tstep
        self.n_samp = n_samp
        self.embd_size = embd_size

        W_val = np.concatenate([ortho_weight(embd_size),
                            ortho_weight(embd_size),
                            ortho_weight(embd_size),
                            ortho_weight(embd_size)], axis=1)
        W = T.shared(value = W_val, name = prefix + '_W', borrow = True)
        U_val = np.concatenate([ortho_weight(embd_size),
                            ortho_weight(embd_size),
                            ortho_weight(embd_size),
                            ortho_weight(embd_size)], axis=1)
        U = T.shared(value = U_val, name = prefix + '_U', borrow = True)
        b_val = np.zeros((4 * embd_size,), dtype = config.floatX)
        b = T.shared(value = b_val, name = prefix + '_b', borrow = True)

        self.input = inp
        self.W = W
        self.U = U
        self.b = b

        # Parallelize: process one batch: n_samp per n_tstep.
        self.inp_t = tensor.dot(self.input, self.W) + self.b
        '''
        rval, updates = T.scan(self._step,
                               sequences    = self.inp_t,
                               outputs_info = [
                                   tensor.zeros_like(self.inp_t),
                                   tensor.zeros_like(self.inp_t)
                                   ],
                               name = prefix + '_layers',
                               n_steps = n_tstep)
        '''

        rval, updates = T.scan(self._step,
                               sequences    = self.inp_t,
                               outputs_info = [
                                   tensor.alloc(
                                       np.asarray(0., dtype = config.floatX),
                                       n_samp,
                                       embd_size
                                       ),
                                   tensor.alloc(
                                       np.asarray(0., dtype = config.floatX),
                                       n_samp,
                                       embd_size
                                       )
                                   ],
                               name = prefix + '_layers',
                               n_steps = n_tstep)

        self.output = rval[0] # h: (n_tstep, n_samp, embd_size)

        self.f = T.function([inp], self.output, name = 'f_' + prefix)
        self.params = [self.W, self.U, self.b]

    def _step(self, x_, h_, c_):
        """
        :x_: W * x_t
        :h_: h_(t-1)
        :c_: c_(t-1)
        """
        preact = x_ + tensor.dot(h_, self.U)

        # Gates, outputs, states
        i = tensor.nnet.sigmoid(self._slice(preact, 0, self.embd_size))
        f = tensor.nnet.sigmoid(self._slice(preact, 1, self.embd_size))
        o = tensor.nnet.sigmoid(self._slice(preact, 2, self.embd_size))
        c = tensor.tanh(self._slice(preact, 3, self.embd_size))

        # States update
        c = f * c_ + i * c

        # Hidden updat
        h = o * tensor.tanh(c)

        return h, c

    def _slice(self, _x, n, dim):
        if _x.ndim == 3:
            return _x[:, :, n * dim:(n + 1) * dim]
        return _x[:, n * dim:(n + 1) * dim]

In [ ]:
class ConvolutionBuilder:
    """
    Net builder for Convolution layer.
    """
    def __init__(self, inp, filter_shape, prefix, stride = (1, 1),
            W = None, b = None, rand_scheme = 'standnormal'):
        n_samp, n_ch, n_row, n_col = inp.shape[0], inp.shape[1], inp.shape[2], inp.shape[3]
        n_filt, n_in_featmap, filt_hgt, filt_wdh = filter_shape

        if W == None:
            if rand_scheme == 'uniform':
                fan_in = n_in_featmap * filt_hgt * filt_wdh
                fan_out = n_filt * filt_hgt * filt_wdh // np.prod(stride)
                W_bound = np.sqrt(6. / (fan_in + fan_out))
                W_val = np.asarray(
                    rng.uniform(
                        low  = -W_bound,
                        high = W_bound,
                        size = filter_shape
                    ),
                    dtype = config.floatX
                )
            elif rand_scheme == 'standnormal':
                W_val = np.asarray(
                    rng.randn(n_filt, n_in_featmap, filt_hgt, filt_wdh),
                    dtype = config.floatX
                )
            elif rand_scheme == 'orthogonal':
                pass
            elif rand_scheme == 'identity':
                pass
            W = T.shared(value = W_val, name = prefix + '_W', borrow = True)

        if b == None:
            b_val = np.zeros((n_filt,), dtype = config.floatX)
            b = T.shared(value = b_val, name = prefix + '_b', borrow = True)

        self.input = inp
        self.W = W
        self.b = b
        self.output = conv2d(input = inp, filters = self.W, filter_shape = filter_shape,
                          subsample = stride) + self.b.dimshuffle('x', 0, 'x', 'x')

        self.f = T.function([inp], self.output, name = 'f_' + prefix)
        self.params = [self.W, self.b]

In [ ]:
# Dense
class DenseBuilder:
    """
    Net builder for Dense layer.
    """
    def __init__(self, inp, in_dim, out_dim, prefix,
                 W = None, b = None, rand_scheme = 'standnormal'):
        n_samp, m = inp.shape[0], inp.shape[1]

        if W == None:
            if rand_scheme == 'uniform':
                W_val = np.asarray(
                    rng.uniform(
                        low  = -np.sqrt(6. / (m + out_dim)),
                        high =  np.sqrt(6. / (m + out_dim)),
                        size = (m, out_dim)
                    ),
                    dtype = config.floatX
                )
            elif rand_scheme == 'standnormal':
                W_val = np.asarray(
                    rng.randn(in_dim, out_dim),
                    dtype = config.floatX
                )
            elif rand_scheme == 'orthogonal':
                pass
            elif rand_scheme == 'identity':
                pass
            W = T.shared(value = W_val, name = prefix + '_W', borrow = True)

        if b == None:
            b_val = np.zeros((out_dim,), dtype = config.floatX)
            b = T.shared(value = b_val, name = prefix + '_b', borrow = True)

        self.input = inp
        self.W = W
        self.b = b
        self.output = tensor.dot(self.input, W) + b

        self.f = T.function([inp], self.output, name = 'f_' + prefix)
        self.params = [self.W, self.b]

### Training
* Loss: categorical crossentropy
* Optimizer: RMSProp
RMSProp is a gradient-based optimization algorithm. It is similar to Adagrad, but introduces an additional decay term to counteract Adagrad’s rapid decrease in learning rate.

The math
$$g_t = \nabla_\theta J( \theta_t )$$
$$E[g^2]_t = \gamma E[g^2]_{t-1} + (1 - \gamma) g_t^2$$
$$RMS[g]_{t} = \sqrt{E[g^2]_t + \epsilon}$$
$$\theta_{t+1} = \theta_t - \dfrac{\eta}{RMS[g]_{t}}f_t$$

We set $\eta = 1e^{-4}$, $\gamma = 0.95$, and $\epsilon = 10e^{-8}$.

* Batch size: 1

In [ ]:
# Implementation of RMSProp
def rmsprop(lr, params, grads, x, y, cost):
    """
    A variant of SGD that scales the step size by running average of the
    recent step norms.

    Parameters
    ----------
    lr : Theano SharedVariable
        Initial learning rate
    pramas: Theano SharedVariable
        Model parameters
    grads: Theano variable
        Gradients of cost w.r.t to parameres
    x: Theano variable
        Model inputs
    y: Theano variable
        Targets
    cost: Theano variable
        Objective fucntion to minimize

    Notes
    -----
    For more information, see [Hint2014]_.

    .. [Hint2014] Geoff Hinton, *Neural Networks for Machine Learning*,
       lecture 6a,
       http://cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf
    """

    zipped_grads = [T.shared(p.get_value() * np.asarray(0., dtype=config.floatX),
                                  name='%s_grad' % str(k))
                    for k, p in zip(range(len(params)), params)]
    running_grads = [T.shared(p.get_value() * np.asarray(0., dtype=config.floatX),
                                  name='%s_rgrad' % str(k))
                    for k, p in zip(range(len(params)), params)]
    running_grads2 = [T.shared(p.get_value() * np.asarray(0., dtype=config.floatX),
                                  name='%s_rgrad2' % str(k))
                      for k, p in zip(range(len(params)), params)]

    zgup = [(zg, g) for zg, g in zip(zipped_grads, grads)]
    rgup = [(rg, 0.95 * rg + 0.05 * g) for rg, g in zip(running_grads, grads)]
    rg2up = [(rg2, 0.95 * rg2 + 0.05 * (g ** 2))
             for rg2, g in zip(running_grads2, grads)]

    f_grad_shared = T.function([x, y], cost,
                               updates=zgup + rgup + rg2up,
                               name='rmsprop_f_grad_shared')

    updir = [T.shared(p.get_value() * np.asarray(0., dtype=config.floatX),
                      name='%s_updir' % k)
             for k, p in zip(range(len(params)), params)]
    updir_new = [(ud, 0.9 * ud - 1e-4 * zg / tensor.sqrt(rg2 - rg ** 2 + 1e-4))
                 for ud, zg, rg, rg2 in zip(updir, zipped_grads, running_grads,
                                            running_grads2)]
    param_up = [(p, p + udn[1])
                for p, udn in zip(params, updir_new)]
    f_update = T.function([lr], [], updates=updir_new + param_up,
                               on_unused_input='ignore',
                               name='rmsprop_f_update')

    return f_grad_shared, f_update

### Implementation Tricks
1. Weights Initialization: The weights are initialized orthogonally.

In [ ]:
# Orthogonal initialization
def ortho_weight(ndim):
    W = np.random.randn(ndim, ndim)
    u, s, v = np.linalg.svd(W)
    return u.astype(config.floatX)

2. Parallel & GPU
3. Batch

## Results
(Run 200 epoches)

| Network | CONV + LSTM | CONV + TDNN |
| ------- |------------:| -----------:|
| Ey      |  81.1299%   |
| Breath  |  86.6417%   |

(Still improving)


# TODO
~~1. run breath~~
2. finish tdnn
3. examine code
4. tune: structure, params, optimizers
5. grant git access
6. look into literature
7. write-up

## Discussion & Thoughts 